In [1]:
import os
import mysql.connector
import pandas as pd
import numpy  as np
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# pip install langchain-google-genai

In [4]:
# pip install langchain

In [5]:
connection = mysql.connector.connect(host='localhost',
                                  user='root',
                                  password='xxxxxxxxxxxxx', # put your sql password here
                                  database='classicmodels')

In [6]:
db_tables=pd.read_sql_query('show tables',connection)
print(db_tables)

  Tables_in_classicmodels
0               customers
1               employees
2                 offices
3            orderdetails
4                  orders
5                payments
6            productlines
7                products


In [7]:
query = "select * from payments"
output = pd.read_sql(query,connection)
print(output.shape)
output

(273, 4)


,customerNumber,checkNumber,paymentDate,amount
0,103,HQ336336,2004-10-19,6066.78
1,103,JM555205,2003-06-05,14571.44
2,103,OM314933,2004-12-18,1676.14
3,112,BO864823,2004-12-17,14191.12
4,112,HQ55022,2003-06-06,32641.98
...,...,...,...,...
268,495,BH167026,2003-12-26,59265.14
269,495,FN155234,2004-05-14,6276.60
270,496,EU531600,2005-05-25,30253.75
271,496,MB342426,2003-07-16,32077.44


In [8]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key='xxxxxxxxxxxxxxxxxxx') # keep your google gemini api key

In [9]:
db = SQLDatabase.from_uri("mysql://root:your_sql_password here@localhost/classicmodels")
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})

In [10]:
print(response)

```sql
SELECT COUNT(*) AS `number of employees`
FROM employees;
```


In [11]:
response = chain.invoke({"question": "what is the Highest payment made by employees"})

In [12]:
print(response)

```sql
SELECT 
    MAX(`amount`) AS `Highest Payment`
FROM 
    `payments`;
```


In [13]:
response = chain.invoke({"question": "write sql query forthe Highest payment made by employees and his name"})

In [14]:
print(response)

```sql
SELECT 
    MAX(amount) AS "Highest Payment Made",
    e.`firstName` || ' ' || e.`lastName` AS "Employee Name"
FROM 
    payments p
JOIN 
    customers c ON p.`customerNumber` = c.`customerNumber`
JOIN 
    employees e ON c.`salesRepEmployeeNumber` = e.`employeeNumber`
GROUP BY 
    e.`employeeNumber`
ORDER BY 
    "Highest Payment Made" DESC
LIMIT 
    5;
```


In [15]:
response = chain.invoke({"question": "Write sql query for fetch highest amount from payement table"})

In [16]:
print(response)

```sql
SELECT MAX(amount) AS "Highest Amount"
FROM payments;
```


In [17]:
query = """SELECT MAX(amount) AS "Highest Amount" FROM payments"""
output = pd.read_sql(query,connection)
print(output.shape)
output

(1, 1)


,Highest Amount
0,120166.58
